# Decision Making in Excitatory & Inhibitory Cells

As our first introduction to two-photon imaging NWB datasets, we'll explore the power of simultaneously imaging hundreds of cells to decode brain activity.

In this notebook, we'll use data from [Najafi et al. (2020)](https://www.cell.com/neuron/fulltext/S0896-6273(19)30848-7) to see if we can use the neural activity in excitatory and inhibitory neurons to predict the animal's decisions.

### About this notebook
This notebook is inspired by [these exercises](https://github.com/vathes/najafi-2018-nwb/blob/master/notebooks/Najafi-2018_example.ipynb) and is still under construction.

<hr>

## Setup

In [3]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pynwb import NWBFile, NWBHDF5IO

print('Packages imported.')

Packages imported.


## Load the data

In [6]:
# This file has been preloaded into the github repository
filename = 'sub-mouse1-fni16_ses-170808180842.nwb'

# Read NWB 2.0 file
nwb_io = NWBHDF5IO(os.path.join(filename), mode = 'r')
nwbfile = nwb_io.read()

First, we'll define a few functions that will help us extract the data we need.

In [17]:
# Extract all trial-based ROI time-series
# ROI time-series has the shape of: (ROI count) x (time instances) x (trial count)`
trial_seg_module = nwbfile.modules.get('Ophys').data_interfaces

def get_trialsegmented_roi_timeseries(event_name, pre_stim_dur, post_stim_dur, trial_seg_module):
    event_roi_timeseries = trial_seg_module.get(event_name).roi_response_series
    tvec = event_roi_timeseries.get('Trial_00').timestamps.value  # timestamps from all trial are the same, so get one from trial_0
    # check if pre/post stim duration is out of bound
    pre_stim_dur = np.maximum(tvec[0], pre_stim_dur)
    post_stim_dur = np.minimum(tvec[-1], post_stim_dur)   
    # extract data
    ix = np.logical_and(tvec >= pre_stim_dur, tvec <= post_stim_dur)
    return np.dstack((d.data.value[:, ix] for d in event_roi_timeseries.values())), tvec[ix]    

In [18]:
# specify event of interest to extract trial data 
segmentation_settings = [
    {'event':'dFoF_initToneAl', 'pre': -10000, 'post': 100},
    {'event':'dFoF_stimAl_noEarlyDec', 'pre': -100, 'post': 10000},
    {'event':'dFoF_firstSideTryAl', 'pre': -250, 'post': 250},
    {'event':'dFoF_rewardAl', 'pre': -250, 'post': 10000}]

# extract trial-based data and average over trial
trial_avg_segments = {}
for setting in segmentation_settings:
    # extract segment
    out = get_trialsegmented_roi_timeseries(setting['event'], setting['pre'], setting['post'], trial_seg_module)
    # average over trial
    trial_avg_segments[setting['event']] = (np.nanmean(out[0], axis=2), out[1])

AttributeError: 'NoneType' object has no attribute 'value'

** Under construction **